# Helmet Detection using YOLOv8

## Change the structure for YOLO Compatibility

In [1]:
import os

# YOLO structure
for split in ['train', 'val', 'test']:
    os.makedirs(f'yolo/images/{split}', exist_ok=True)
    os.makedirs(f'yolo/labels/{split}', exist_ok=True)

print("✅ YOLOv8 folder structure created!")


✅ YOLOv8 folder structure created!


In [2]:
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import shutil

ANNOTATIONS_DIR = 'annotations'
IMAGES_DIR = 'images'

def convert_bbox_to_yolo(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x_center = (box[0] + box[2]) / 2.0 * dw
    y_center = (box[1] + box[3]) / 2.0 * dh
    width = (box[2] - box[0]) * dw
    height = (box[3] - box[1]) * dh
    return (x_center, y_center, width, height)

# Collect all file names
all_files = [f for f in os.listdir(ANNOTATIONS_DIR) if f.endswith('.xml')]

# Split into train/val/test
train_files, test_files = train_test_split(all_files, test_size=0.2, random_state=42)
val_files, test_files = train_test_split(test_files, test_size=0.5, random_state=42)
splits = {'train': train_files, 'val': val_files, 'test': test_files}


In [3]:
for split, files in splits.items():
    for file in tqdm(files, desc=f"Processing {split} set"):
        image_id = file.replace('.xml', '')
        xml_path = os.path.join(ANNOTATIONS_DIR, file)
        img_path = os.path.join(IMAGES_DIR, image_id + ".png")
        output_txt_path = f"yolo/labels/{split}/{image_id}.txt"
        output_img_path = f"yolo/images/{split}/{image_id}.png"

        tree = ET.parse(xml_path)
        root = tree.getroot()
        size = root.find('size')
        w = int(size.find('width').text)
        h = int(size.find('height').text)

        with open(output_txt_path, "w") as f:
            for obj in root.findall('object'):
                label = obj.find('name').text
                if label == "With Helmet":
                    class_id = 0  # Only one class
                else:
                    continue  # Skip unknown classes
                bndbox = obj.find('bndbox')
                xmin = int(bndbox.find('xmin').text)
                ymin = int(bndbox.find('ymin').text)
                xmax = int(bndbox.find('xmax').text)
                ymax = int(bndbox.find('ymax').text)
                yolo_bbox = convert_bbox_to_yolo((w, h), (xmin, ymin, xmax, ymax))
                f.write(f"{class_id} " + " ".join(f"{x:.6f}" for x in yolo_bbox) + "\n")

        # Copy image to split folder
        shutil.copy(img_path, output_img_path)


Processing test set: 100%|█████████████████████████████████████████████████████████████| 77/77 [00:01<00:00, 39.51it/s]


In [4]:
yaml_text = """
train: yolo/images/train
val: yolo/images/val
test: yolo/images/test

nc: 1
names: ['helmet']
"""

with open("yolo/data.yaml", "w") as f:
    f.write(yaml_text.strip())

print("✅ data.yaml created!")


✅ data.yaml created!


In [6]:
import os

base_path = os.path.abspath("yolo")

data_yaml = f"""
train: {os.path.join(base_path, 'images/train').replace(os.sep, '/')}
val: {os.path.join(base_path, 'images/val').replace(os.sep, '/')}
test: {os.path.join(base_path, 'images/test').replace(os.sep, '/')}

nc: 1
names: ['helmet']
"""

with open(os.path.join(base_path, "data.yaml"), "w") as f:
    f.write(data_yaml.strip())

print("✅ data.yaml updated with absolute paths!")


✅ data.yaml updated with absolute paths!


## Train YOLOv8 on the dataset
